In [2]:
import pandas as pd
import os
import json
import pymysql
from pyecharts.charts import Graph
from pyecharts import options as opts
from pyecharts.globals import ThemeType
import re
from dominate.tags import *
import dominate
import difflib
from pyecharts.commons.utils import JsCode

In [3]:
db = pymysql.connect(host='localhost',
                     user='',
                     password='',
                     database='public')
cursor = db.cursor()

In [4]:
lang_map = {}
line_map = {}
with open("zh_CN.lang", "r", encoding="utf-8") as lang:
    for i in lang:
        i = re.sub("§.", "", i)
        i = i.replace("%n", "")
        if "gtnh.line" in i:
            temp = i.split("=", maxsplit=1)
            line_map[temp[0].replace("gtnh.line", "")] = temp[1].strip()
        elif "name" in i or "desc" in i:
            temp = i.split("=", maxsplit=1)
            lang_map[temp[0].replace("gtnh.quest", "")] = temp[1].strip()

In [5]:
df = pd.DataFrame(columns=["quest_file_name", "quest_id",
                  "quest_name", "quest_desc", "quest_icon", "pre_quests"])
for quest_line in os.listdir("DefaultQuests/Quests/"):
    for quest in os.listdir(f"DefaultQuests/Quests/{quest_line}/"):
        que = json.load(
            open(f"DefaultQuests/Quests/{quest_line}/{quest}", "r", encoding="utf-8"))
        
        pre_quests = []
        for i in que["preRequisites:9"]:
            pre_quests.append(que["preRequisites:9"][i]["questIDLow:4"])
        
        item_id = "i~" + que["properties:10"]['betterquesting:10']["icon:10"]["id:8"].replace(":", "~") + "~" + str(que["properties:10"]['betterquesting:10']["icon:10"]["Damage:2"])
        cursor.execute(f"select ID, INTERNAL_NAME, LOCALIZED_NAME from item where ID like '%{item_id}%'")
        icon_name = ""
        try:
            icon_name = cursor.fetchall()[0][2].encode(
                "utf-8").decode("unicode_escape")
        except:
            ...
        df.loc[len(df)] = [quest, que["questIDLow:4"], que["properties:10"]
                           ["betterquesting:10"]["name:8"], que["properties:10"]["betterquesting:10"]["desc:8"], icon_name, pre_quests]

counter = 1
for i in df[df["quest_name"] == "AE Spatial Storage"].sort_values("quest_id").index:
    df.loc[i, "quest_name"] = df.loc[i, "quest_name"] + str(counter)
    counter += 1

df["quest_name_zh"] = df.apply(lambda x: lang_map[str(
    x["quest_id"])+".name"] if str(x["quest_id"])+".name" in lang_map.keys() else "", axis=1)
df["quest_desc_zh"] = df.apply(lambda x: lang_map[str(
    x["quest_id"])+".desc"] if str(x["quest_id"])+".desc" in lang_map.keys() else "", axis=1)

repeats = ["AE空间存储", "自动合成", "种子袋",
           "放射性同位素温差发电机", "杀戮时刻(岩浆怪)", "激发流体核弹", "聚四氟乙烯"]
for word in repeats:
    counter = 1
    for i in df[df["quest_name_zh"] == word].sort_values("quest_id").index:
        df.loc[i, "quest_name_zh"] = df.loc[i, "quest_name_zh"] + str(counter)
        counter += 1

In [29]:
for questline in os.listdir("DefaultQuests/QuestLines"):
    nodes = []
    links = []
    for quest in os.listdir(f"DefaultQuests/QuestLines/{questline}"):
        if quest == "QuestLine.json":
            continue
        que = json.load(
            open(f"DefaultQuests/QuestLines/{questline}/{quest}", "r", encoding="utf-8"))
        line = df[df["quest_id"] == que["questIDLow:4"]]

        desc = line["quest_desc"].values[0]
        if len(line["quest_desc_zh"].values[0]) != 0:
            desc = line["quest_desc_zh"].values[0]

        temp_list = []
        for i in range(0, len(desc), 50):
            temp_list.append(desc[i : i + 50])

        if os.path.exists(f"html/2.7.0全物品图标/2.7.0/{line["quest_icon"].values[0]}.png"):
            path = f"image://2.7.0全物品图标/2.7.0/{line["quest_icon"].values[0]}.png"
        else:
            path = ""
        
        name = line["quest_name"].values[0]
        if len(line["quest_name_zh"].values[0]) != 0:
            name = line["quest_name_zh"].values[0]

        new_node = {"name": name,
                        "symbolSize": int(que["sizeX:3"]) * 1.3,
                        "symbol": path,
                        "x": (que["x:3"] + int(que["sizeX:3"]) / 2),
                        "y": (que["y:3"] + int(que["sizeX:3"]) / 2),
                        "tooltip": "<br/>".join(temp_list)
                        }
        nodes.append(new_node)
        pre_quests = line["pre_quests"].values[0]
        for quest_id in pre_quests:
            pre_que = df[df["quest_id"] == quest_id]

            pre_name = pre_que["quest_name"].values[0]
            if len(pre_que["quest_name_zh"].values[0]) != 0:
                pre_name = pre_que["quest_name_zh"].values[0]

            links.append({"source": pre_name,
                          "target": name})

    g = (
        Graph(init_opts=opts.InitOpts(width="100%",
              height="900%", bg_color="#f5f0d3", chart_id="this_chart"))
        .add("", nodes, links, is_draggable=False, edge_symbol=['circle', 'arrow'], repulsion=0, label_opts=opts.LabelOpts(position="up", is_show=False), gravity=0, layout="none", edge_symbol_size=10)
        .add_js_funcs(
        """
            chart_this_chart.on('click', function (params) {
                    if (params.dataType === 'node') {
                        navigator.clipboard.writeText(params.data.tooltip.replaceAll("<br/>", ""));
                    }
            });
        """
        )
        .render(f"html/{questline.split("-")[0]}.html")
    )
    temp = []
    counter = 0
    for i in nodes:
        if i["name"] in temp:
            print(questline.split("-")[0], i["name"], counter)
            counter += 1
        else:
            temp.append(i["name"])

In [26]:
temp = []
counter = 0
for i in all_nodes:
    if i["name"] in temp:
        print(i["name"], counter)
        counter += 1
    else:
        temp.append(i["name"])

僧侣蜂 0
种子袋 1
萤石 2
符文 3
我不去就山,让山来就我 4
浸渍框架 5
放射性同位素温差发电机 6
杀戮时刻(岩浆怪) 7
巫术种子 8
赌上蒸馏 9
工业3D打印机 10
工业焦炉 11
作物加速 12
精密组装机 13
点亮它! 14
钢叶 15
这啥玩意儿？ 16
激发流体核弹 17
高温超级增殖反应堆 18
无尽涡轮 19
2M涡轮 20
单涡轮UV发电 21
弧光灯 22
发射控制器 23
晶圆 24
别把手指插进插座里 25
电路基板 26
§3§lMultiblock Revolution 27
电阻 28
安全第一 29
更多蒸汽! 30
§3§lSteel Chest? 31
欢迎来玩林业MOD 32
工作台2.0 33
一种更快捷的出行方式 34
§2§lCompact Storage 35
杂酚油 36
更加更加紧凑的存储 37
更加紧凑的存储 38
农场"智者"手册 39
化学终结者 40
戴森球地面单元 41
升级永恒线圈 42
升级海珀珍线圈 43
3阶纳米锻炉 44
3阶PCB工厂 45
以太之力 46
Thaumometric Essentia Cell 47
Waterproof Tech 48
我应该在哪建造戴森球? 49
空冻塔 50
电池合金 51
线缆二极管 52
大马士革钢 53
钻石箱子...哦,闪闪发亮 54
二极管 55
电路基板 56
引爆液体获取能量 57
胶水 58
重燃油 59
氢气、甲烷或者一些屁(天然气) 60
轻燃油 61
润滑油 62
甲烷 63
石脑油 64
硫酸 65
生物柴油 66
§5§6§lDark Steel Chest 67
不锈钢管道 68
塑料电路基板 69
乙醇 70
发酵生物质 71
集成逻辑电路 72
氯化铁 73
甲烷 74
太阳能 75
太阳之力(ULV)1x1 76
太阳之力(ULV) 77
氢氧化钠 78
过硫酸钠 79
汞太多了? 80
晶体管 81
散热 82
末影之眼 83
动能发电机(HV) 84
水力动能发生机(HV) 85
风力动能发生机(HV) 86
需要一个地方来存放所有这些矿石？ 87
转子 88
将能量送回 89
摇起来... 90
聚四氟乙烯 91
风力计 92
合适的能源 93
基础核燃料:钍 94
二联燃料棒,双倍的快乐(doub

In [61]:
quest_line_order = []
for line in open("DefaultQuests/QuestLinesOrder.txt").readlines():
    quest_line_order.append(line.split(": ")[1].strip())

In [74]:
len(quest_line_order)

47

In [75]:
a = ul()
    # if "html" in i:
    #     a += li(button(i.split(".html")[0], id=i))
for que_line in quest_line_order:
    q = difflib.get_close_matches(que_line, os.listdir("html"))[0]
    a += li(button(q.split(".html")[0], id=q))
print(a)

<ul>
  <li>
    <button id="AndSoItBegins.html">AndSoItBegins</button>
  </li>
  <li>
    <button id="TipsandTricks.html">TipsandTricks</button>
  </li>
  <li>
    <button id="Tier0StoneAge.html">Tier0StoneAge</button>
  </li>
  <li>
    <button id="Tier05Steam.html">Tier05Steam</button>
  </li>
  <li>
    <button id="Tier1LV.html">Tier1LV</button>
  </li>
  <li>
    <button id="Tier2MV.html">Tier2MV</button>
  </li>
  <li>
    <button id="Tier3HV.html">Tier3HV</button>
  </li>
  <li>
    <button id="Tier4EV.html">Tier4EV</button>
  </li>
  <li>
    <button id="Tier5IV.html">Tier5IV</button>
  </li>
  <li>
    <button id="Tier6LuV.html">Tier6LuV</button>
  </li>
  <li>
    <button id="Tier7ZPM.html">Tier7ZPM</button>
  </li>
  <li>
    <button id="Tier8UV.html">Tier8UV</button>
  </li>
  <li>
    <button id="Tier9UHV.html">Tier9UHV</button>
  </li>
  <li>
    <button id="Tier10UEV.html">Tier10UEV</button>
  </li>
  <li>
    <button id="Tier11UIV.html">Tier11UIV</button>
  </li>
  <li>


42

In [4]:
str = """
        document.getElementById('%s').addEventListener('click', function () {
            document.getElementById('contentFrame').src = '%s';
        });
"""
for i in os.listdir("html"):
    if "html" in i:
        print(str % (i, "html/" + i))


        document.getElementById('AdeptThaumaturgy.html').addEventListener('click', function () {
            document.getElementById('contentFrame').src = 'html/AdeptThaumaturgy.html';
        });


        document.getElementById('AndSoItBegins.html').addEventListener('click', function () {
            document.getElementById('contentFrame').src = 'html/AndSoItBegins.html';
        });


        document.getElementById('AppliedEnergisti.html').addEventListener('click', function () {
            document.getElementById('contentFrame').src = 'html/AppliedEnergisti.html';
        });


        document.getElementById('BasicAutomation.html').addEventListener('click', function () {
            document.getElementById('contentFrame').src = 'html/BasicAutomation.html';
        });


        document.getElementById('BeeBreeding.html').addEventListener('click', function () {
            document.getElementById('contentFrame').src = 'html/BeeBreeding.html';
        });


        document.getEl